# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Modify-ecoinvent" data-toc-modified-id="Modify-ecoinvent-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modify ecoinvent</a></div><div class="lev1 toc-item"><a href="#Prepare-additional-datasets" data-toc-modified-id="Prepare-additional-datasets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare additional datasets</a></div><div class="lev2 toc-item"><a href="#import-from-excel" data-toc-modified-id="import-from-excel-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>import from excel</a></div><div class="lev1 toc-item"><a href="#Define-REMIND-scenarios-and-years-to-be-used:" data-toc-modified-id="Define-REMIND-scenarios-and-years-to-be-used:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define REMIND scenarios and years to be used:</a></div><div class="lev1 toc-item"><a href="#Modify-all-electricity-generation-and-market-datasets:" data-toc-modified-id="Modify-all-electricity-generation-and-market-datasets:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modify all electricity generation and market datasets:</a></div><div class="lev1 toc-item"><a href="#create-current-version-of-ecoinvent-with-additional-technologies-added-in:" data-toc-modified-id="create-current-version-of-ecoinvent-with-additional-technologies-added-in:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>create current version of ecoinvent with additional technologies added in:</a></div>

# Modify ecoinvent

In this notebook we actually run the functions that we've been creating so that we can modify ecoinvent!

In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
%matplotlib inline

Import useful libraries and set project:

In [13]:
%run initialize_notebook.ipynb
from helpers import lcahelp, eimod, ei2rmnd

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

# Prepare additional datasets

We import some additional datasets for carbon capture and storage electricity. These have been prepared in Simapro and exported. We have cleaned up the datasets and stored them in excel using Brightway input output functionality. We read them from excel here:

## import from excel

In [5]:
lcahelp.import_karma()

Database Carma CCS already present.


# Define REMIND scenarios and years to be used:

Here we set the scenarios and years for which we want to use when creating a new version of ecoinvent:

In [6]:
database_dict = collections.defaultdict(dict)

for year in [2015, 2030, 2050]:
#for year in [2015]:
    for scenario in ['BAU','RCP26','RCP37']:
        if year == 2015 and scenario != 'BAU':
            continue
        db_name = 'ecoinvent_Remind_' + scenario + '_' + str(year)
        database_dict[db_name] = {'year' : year , 'scenario' : scenario}

#database_dict['test'] = {'year' : 2041 , 'scenario' : 'BAU'}

In [7]:
database_dict

defaultdict(dict,
            {'ecoinvent_Remind_BAU_2015': {'year': 2015, 'scenario': 'BAU'},
             'ecoinvent_Remind_BAU_2030': {'year': 2030, 'scenario': 'BAU'},
             'ecoinvent_Remind_RCP26_2030': {'year': 2030,
              'scenario': 'RCP26'},
             'ecoinvent_Remind_RCP37_2030': {'year': 2030,
              'scenario': 'RCP37'},
             'ecoinvent_Remind_BAU_2050': {'year': 2050, 'scenario': 'BAU'},
             'ecoinvent_Remind_RCP26_2050': {'year': 2050,
              'scenario': 'RCP26'},
             'ecoinvent_Remind_RCP37_2050': {'year': 2050,
              'scenario': 'RCP37'}})

In [8]:
databases

Databases dictionary with 12 object(s):
	Carma CCS
	Inventory flows
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2015
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050
	ecoinvent_added_technologies

# Modify all electricity generation and market datasets:

This cell uses the Wurst functionality as well as the functions that we wrote in the notebook "_Functions_to_modify_ecoinvent.ipynb" to make the changes to ecoinvent:

In [ ]:
eimod.modify_electricity_generation_datasets(database_dict)

Getting activity data


# create current version of ecoinvent with additional technologies added in:

In [ ]:
key = 'ecoinvent_added_technologies'
    
if key not in databases:    
    
    db = extract_brightway2_databases(['Carma CCS','ecoinvent_3.5'])
    default_global_location(db)
    eimod.fix_unset_technosphere_and_production_exchange_locations(db)
    #set_global_location_for_additional_datasets(db)
    eimod.remove_nones(db)
    eimod.rename_locations(db, eimod.fix_names)
    eimod.add_negative_CO2_flows_for_biomass_CCS(db)

    eimod.rename_locations(db, eimod.fix_names_back)
    if key in databases:
        del databases[key]

    write_brightway2_database(db, key)

    lca = LCA({Database(key).random(): 1}, method=lcia_methods['CC'])
    lca.lci()
    lca.lcia()
    print("Test LCA Score: {}".format(lca.score))

else: print('Database already exists')